In [157]:
!pip install "sagemaker>=2.31.0" "transformers==4.4.2" "datasets[s3]==1.5.0" --upgrade

  Using cached boto3-1.16.43-py2.py3-none-any.whl (130 kB)
  Using cached botocore-1.19.43-py2.py3-none-any.whl (7.2 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/urllib3-1.26.4.dist-info/METADATA'



In [158]:
%%capture
import IPython
!conda install -c conda-forge ipywidgets -y
IPython.Application.instance().kernel.do_shutdown(True) # has to restart kernel so changes are used

In [1]:
import sagemaker.huggingface
import pandas as pd
import boto3
import io
from io import StringIO
from sagemaker import get_execution_role
import pickle
# using the autotokenizer from huggingface transformer 
from transformers import AutoTokenizer

In [2]:

# I am checking the transformer with the dummy file 
data = pd.read_csv('/home/ec2-user/SageMaker/notebooks/lighthouse/data/labelled_statement.csv')
data.head(5)
data_set = data.text
tokenizer = AutoTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")


In [3]:
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:700], data_set))
train_tokens

[['[CLS]',
  'The',
  'Ġroot',
  'Ġcause',
  'Ġof',
  'Ġthis',
  'Ġevent',
  'Ġis',
  'Ġthat',
  'Ġvalve',
  'ĠXV',
  '71',
  '50',
  '.',
  '46',
  'Ġwas',
  'Ġstuck',
  'Ġopen',
  'Ġ(',
  'bottom',
  'Ġseat',
  'Ġposition',
  ')',
  'Ġon',
  'Ġmanifold',
  'ĠM',
  '71',
  '50',
  '.',
  '18',
  '.'],
 ['[CLS]',
  'While',
  'Ġinspecting',
  'Ġequipment',
  'Ġit',
  'Ġwas',
  'Ġnoticed',
  'Ġthat',
  'Ġvalve',
  'ĠXV',
  '71',
  '50',
  '.',
  '46',
  'Ġon',
  'Ġmanifold',
  'ĠM',
  '71',
  '50',
  '.',
  '18',
  'Ġwas',
  'Ġstuck',
  'Ġin',
  'Ġthe',
  'Ġopen',
  '.'],
 ['[CLS]',
  'F',
  'INAL',
  'ĠCA',
  'USE',
  'ĠAN',
  'AL',
  'YS',
  'IS',
  ':',
  'ĠThe',
  'Ġcause',
  'Ġof',
  'Ġthis',
  'Ġevent',
  'Ġis',
  'Ġthat',
  'Ġvalve',
  'ĠXV',
  '71',
  '50',
  '.',
  '46',
  'Ġwas',
  'Ġstuck',
  'Ġopen',
  'Ġ(',
  'bottom',
  'Ġseat',
  'Ġposition',
  ')',
  'Ġon',
  'Ġmanifold',
  'ĠM',
  '71',
  '50',
  '.',
  '18',
  '.'],
 ['[CLS]',
  'It',
  'Ġwas',
  'Ġnoted',
  'Ġby',
  '

In [4]:
# As we checked it is working it is time for us to use it on the s3 file




In [5]:
bucket = 'deviation-intelligence-f8b2af20'
subfolder = 'tmp/test_aws_pipeline/state_1/'
# getting the execution role to download the file
role = get_execution_role()

In [6]:
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']
# checking the content of the file
for f in contents:
    print(f['Key'])

tmp/test_aws_pipeline/state_1/
tmp/test_aws_pipeline/state_1/1613953.txt
tmp/test_aws_pipeline/state_1/1617828.txt
tmp/test_aws_pipeline/state_1/1630812.txt
tmp/test_aws_pipeline/state_1/1702536.txt
tmp/test_aws_pipeline/state_1/1702884.txt
tmp/test_aws_pipeline/state_1/1709688.txt
tmp/test_aws_pipeline/state_1/1711992.txt
tmp/test_aws_pipeline/state_1/1729673.txt
tmp/test_aws_pipeline/state_1/1730034.txt
tmp/test_aws_pipeline/state_1/1730054.txt
tmp/test_aws_pipeline/state_1/1730077.txt
tmp/test_aws_pipeline/state_1/1730109.txt
tmp/test_aws_pipeline/state_1/1730124.txt
tmp/test_aws_pipeline/state_1/1730182.txt
tmp/test_aws_pipeline/state_1/1730185.txt
tmp/test_aws_pipeline/state_1/1730484.txt
tmp/test_aws_pipeline/state_1/1730517.txt
tmp/test_aws_pipeline/state_1/1730519.txt
tmp/test_aws_pipeline/state_1/1730880.txt
tmp/test_aws_pipeline/state_1/1731144.txt
tmp/test_aws_pipeline/state_1/1731333.txt
tmp/test_aws_pipeline/state_1/1732779.txt
tmp/test_aws_pipeline/state_1/1733019.txt
tmp

In [7]:
my_bucket = 'deviation-intelligence-f8b2af20'
my_file = 'tmp/test_aws_pipeline/state_1/1613953.txt'
s3client = boto3.client('s3')
response = s3client.get_object(Bucket=my_bucket, Key=my_file)
body = response['Body']



In [8]:
# we can select a file to do tokenization as 1613953.txt
S3_Key='tmp/test_aws_pipeline/state_1/1613953.txt'
client = boto3.client('s3')
s3_object = client.get_object(Bucket=bucket, Key=S3_Key)
df = pd.read_csv(io.BytesIO(s3_object['Body'].read()), error_bad_lines=False,names=['LABEL'])

In [9]:
#Chekcing for the content is copued correctly for the transformation 
df.LABEL.head(5)

0                                       event_summary 
1                                   Problem statement:
2    1. TT4 logger (GDC4N07XA0) failed to read the ...
3    2. TT4 logger reading number (GDP4N04EW0) on T...
4                                               Scope:
Name: LABEL, dtype: object

In [10]:
# tokenizing using the above transformer tokenizer
# here are I have taqken maximum length of each sentence as more than 200 words it can be reduced further
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:200], df.LABEL))

In [11]:
#chekcing  if tokens are there 
train_tokens


[['[CLS]', 'event', '_', 'summary', 'Ġ'],
 ['[CLS]', 'Problem', 'Ġstatement', ':'],
 ['[CLS]',
  '1',
  '.',
  'ĠTT',
  '4',
  'Ġlogger',
  'Ġ(',
  'G',
  'DC',
  '4',
  'N',
  '07',
  'X',
  'A',
  '0',
  ')',
  'Ġfailed',
  'Ġto',
  'Ġread',
  'Ġthe',
  'Ġdata',
  ';'],
 ['[CLS]',
  '2',
  '.',
  'ĠTT',
  '4',
  'Ġlogger',
  'Ġreading',
  'Ġnumber',
  'Ġ(',
  'G',
  'DP',
  '4',
  'N',
  '04',
  'EW',
  '0',
  ')',
  'Ġon',
  'ĠT',
  'DS',
  'Ġwas',
  'Ġinconsistent',
  'Ġwith',
  'Ġthe',
  'Ġmarking',
  'Ġnumber',
  'Ġon',
  'Ġthe',
  'Ġdevice',
  'Ġ(',
  'G',
  'DC',
  '4',
  'N',
  '06',
  'G',
  'X',
  '0',
  ')'],
 ['[CLS]', 'Scope', ':'],
 ['[CLS]', 'Delivery', 'ĠNo', ':', 'Ġ12', '103', '28', '218'],
 ['[CLS]',
  'Material',
  'ĠNumber',
  ':',
  'Ġ101',
  '8',
  '65',
  '18',
  'ï',
  '¼',
  'Ī',
  'by',
  'ĠAir',
  'fre',
  'ight',
  'ï',
  '¼',
  'ī'],
 ['[CLS]',
  'Material',
  'ĠName',
  ':',
  'ĠMad',
  'op',
  'ar',
  'Ġ200',
  '/',
  '50',
  'ĠGran',
  'ules',
  'Ġfor',

In [12]:
# As tokens are converted into list converitng them back to dataframe
train_tokens=pd.DataFrame(train_tokens)
train_tokens

,0,1,2,3,4,5,6,7,8,9,...,186,187,188,189,190,191,192,193,194,195
0,[CLS],event,_,summary,Ġ,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,[CLS],Problem,Ġstatement,:,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,[CLS],1,.,ĠTT,4,Ġlogger,Ġ(,G,DC,4,...,None,None,None,None,None,None,None,None,None,None
3,[CLS],2,.,ĠTT,4,Ġlogger,Ġreading,Ġnumber,Ġ(,G,...,None,None,None,None,None,None,None,None,None,None
4,[CLS],Scope,:,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,[CLS],Delivery,ĠNo,:,Ġ12,103,28,218,None,None,...,None,None,None,None,None,None,None,None,None,None
6,[CLS],Material,ĠNumber,:,Ġ101,8,65,18,ï,¼,...,None,None,None,None,None,None,None,None,None,None
7,[CLS],Material,ĠName,:,ĠMad,op,ar,Ġ200,/,50,...,None,None,None,None,None,None,None,None,None,None
8,[CLS],A,ctions,Ġtaken,:,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,[CLS],-,ĠQ,SM,Ġwas,Ġinformed,Ġfor,Ġacknowledgment,Ġand,Ġtrend,...,None,None,None,None,None,None,None,None,None,None


In [13]:

serializedListObject = pickle.dumps(train_tokens)
S3_Key='tmp/test_aws_pipeline/state_2/1613953.csv'
response = client.put_object( 
    Bucket=bucket,
    Body= serializedListObject,
    Key=S3_Key,
    ServerSideEncryption = 'aws:kms'
    )

#x-amz-server-side-encryption